In [ ]:
import spacy
from spacy import displacy 
nlp = spacy.load('en_core_web_sm')
# Text with nlp
doc = nlp(" Multiple tornado warnings were issued for parts of New York on Sunday night.The first warning, which expired at 9 p.m., covered the Bronx, Yonkers and New Rochelle. More than 2 million people live in the impacted area.")
# Display Entities
displacy.render(doc, style="ent")

'<div class="entities" style="line-height: 2.5; direction: ltr"> Multiple tornado warnings were issued for parts of \n<mark class="entity" style="background: #feca74; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    New York\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; vertical-align: middle; margin-left: 0.5rem">GPE</span>\n</mark>\n on \n<mark class="entity" style="background: #bfe1d9; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    Sunday night\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; vertical-align: middle; margin-left: 0.5rem">TIME</span>\n</mark>\n.The \n<mark class="entity" style="background: #e4e7d2; padding: 0.45em 0.6em; margin: 0 0.25em; line-height: 1; border-radius: 0.35em;">\n    first\n    <span style="font-size: 0.8em; font-weight: bold; line-height: 1; border-radius: 0.35em; vertical-align: mi

In [ ]:
!python -m spacy download xx_ent_wiki_sm

2023-04-22 05:01:40.902099: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 55.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('xx_ent_wiki_sm')


In [4]:
nlp_wk = spacy.load('xx_ent_wiki_sm')
doc = nlp_wk("Multiple tornado warnings were issued for parts of New York on Sunday night.The first warning, which expired at 9 p.m., covered the Bronx, Yonkers and New Rochelle. More than 2 million people live in the impacted area.")

OSError: ignored

In [ ]:
doc.ents

(New York, Bronx, Yonkers, New Rochelle)

In [2]:
import pandas as pd
import numpy as np

# Define the number of tweets to generate
num_tweets = 100

# Define a list of disaster keywords
disaster_keywords = ["earthquake", "flood", "fire", "tornado", "hurricane", "tsunami"]

# Define a list of locations
cities = ["Los Angeles", "New York", "San Francisco", "Chicago", "Miami", "Seattle"]
countries = ["United States", "Canada", "Mexico", "Brazil", "China", "India"]
counties = ["Cook County", "Los Angeles County", "Miami-Dade County", "Harris County", "Maricopa County", "San Diego County"]
locations = cities + countries + counties

# Generate random tweets and locations
tweets = []
tweet_locations = []
for i in range(num_tweets):
    location=np.random.choice(locations)
    tweet = "There was a " + np.random.choice(disaster_keywords) + " in " + location + " today."
    tweets.append(tweet)
    tweet_locations.append(location)

# Create a dataframe from the tweets and locations
df = pd.DataFrame({'Tweet': tweets, 'Location': tweet_locations})

# Write the dataframe to a CSV file
df.to_csv('sample_disaster_tweets.csv', index=False)


In [3]:
import pandas as pd
import spacy

nlp_wk = spacy.load('xx_ent_wiki_sm')

csv_path = "disaster_tweets.csv"

# Read the CSV file into a pandas dataframe
df = pd.read_csv(csv_path)

# Define the name of the text column in the dataframe
text_column = "Tweet"

# Create an empty list to store the locations for each row
locations = []

# Iterate over each row in the dataframe
for index, row in df.iterrows():
    # Extract the text from the specified column
    text = row[text_column]
    # Use spaCy to identify all the locations in the text
    doc = nlp_wk(text)
    locs = [[index, ent.text] for ent in doc.ents if ent.label_ == 'LOC']
    # Append the list of locations to the main locations list
    locations.extend(locs)

# Convert the locations list to a dataframe
locations_df = pd.DataFrame(locations, columns=["row_index", "location"])



# # Write the result to a new CSV file
# result.to_csv("output.csv", index=False)


OSError: ignored

In [ ]:
locations_df

In [ ]:
!pip install geopandas

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.1/16.1 MB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 24.1 MB/s eta 0:00:00


In [ ]:
import pandas as pd 
import geopandas as gpd 
import geopy 
import matplotlib.pyplot as plt
from geopy.extra.rate_limiter import RateLimiter
locator = geopy.geocoders.Nominatim(user_agent="mygeocoder")
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
locations_df["address"] = locations_df["location"].apply(geocode)

In [ ]:
locations_df['coordinates'] = locations_df['address'].apply(lambda loc: tuple(loc.point) if loc else None)
locations_df[['latitude', 'longitude', 'altitude']] = pd.DataFrame(locations_df['coordinates'].tolist(), index=df.index)
locations_df.latitude.isnull().sum()
locations_df = locations_df[pd.notnull(locations_df["latitude"])]

In [ ]:
!pip install folium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import folium
from folium.plugins import FastMarkerCluster
folium_map = folium.Map(location=[59.338315,18.089960],
 zoom_start=2,
 tiles='CartoDB dark_matter')
FastMarkerCluster(data=list(zip(locations_df['latitude'].values, locations_df['longitude'].values))).add_to(folium_map)
folium.LayerControl().add_to(folium_map)
folium_map